In [1]:
!pip install faiss-cpu
!pip install sentence-transformers
!pip install tiktoken
!pip install langchain
!pip install -U langchain-openai
!pip install -U langchain-community
!pip install openpyxl


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
   -------------------------- ------------- 41.0/61.2 kB ? eta -:--:--
   ---------------------------------------- 61.2/61.2 kB 821.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/418.7 kB ? eta -:--:--
   ----- ---------------------------------- 61.4/418.7 kB 1.7 MB/s eta 0:00:01
   ---------- ----------------------------- 112.6/418.7 kB 1.3 MB/s eta 0:00:01
   ---------------- ----------------------- 174.1/418.7 kB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 204.8/418.7 kB 1.1 MB/s eta 0:00:01
   -------------------------- ------------- 276.5/418.7 kB 1.3 MB/s eta 0:00:01
   ------------------------------ --------- 317.4/418.7 kB 1.2 MB/s eta 0:00:01
   ----------------------------------- ---- 368.6/418.7 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 418.7/418.7 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/606.1 kB ? eta -:--:-

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
browser-use 0.1.37 requires langchain==0.3.14, but you have langchain 0.3.20 which is incompatible.
browser-use 0.1.37 requires langchain-openai==0.3.1, but you have langchain-openai 0.3.10 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.5 MB 100.9 kB/s eta 0:00:25
   ---------------------------------------- 0.0/2.5 MB 100.9 kB/s eta 0:00:25
    --------------------------------------- 0.0/2.5 MB 93.7 kB/s eta 0:00:27
    ----

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
browser-use 0.1.37 requires langchain==0.3.14, but you have langchain 0.3.21 which is incompatible.
browser-use 0.1.37 requires langchain-openai==0.3.1, but you have langchain-openai 0.3.10 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
import os
import pandas as pd
import requests
from dotenv import load_dotenv

# LangChain 関連
from langchain.chains import RetrievalQA
from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain.schema import Document

# GraphRAG用
import torch
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration
import networkx as nx

In [16]:
# .env ファイルを読み込む
load_dotenv()

True

In [4]:
# 1. データの読み込み
df = pd.read_excel("100データ5.xlsx", engine="openpyxl")
df.head()

,id,date,timestamp,place,latitude,longitude,lat_long,diary_text
0,1,18278,1950-01-15T10:30:00Z,大阪市住吉区,34.5938,135.4937,"34.5938° N, 135.4937°",父の数学の参考書を読みながら、将来の夢について考えた。野球への情熱が日に日に強くなっていく。...
1,2,18709,1951-03-22T15:45:00Z,大阪市住吉区,34.5938,135.4937,"34.5938° N, 135.4937°",今日の野球の練習は厳しかったが、チームメイトと共に汗を流すことで絆が深まった。夕方は家族と一...
2,3,19124,1952-05-10T09:15:00Z,大阪市住吉区,34.5938,135.4937,"34.5938° N, 135.4937°",今朝、父と一緒に野球の戦術について語り合った。数学的思考が野球にも活かせると感じた。夕方は近...
3,4,19565,1953-07-25T16:20:00Z,大阪市住吉区,34.5938,135.4937,"34.5938° N, 135.4937°",夏の甲子園予選に向けて、猛暑の中で練習に励んでいる。汗と熱意で、チームの勝利を信じている。夜...
4,5,19972,1954-09-05T14:40:00Z,大阪市住吉区,34.5938,135.4937,"34.5938° N, 135.4937°",高校野球の最後の大会。悔いなく全力で戦った。負けは悔しいが、チームメイトとの絆は一生の宝物と...


In [6]:
API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")  # 直接記述する場合は "your-api-key" に変更

# 緯度・経度から住所を取得し、国内・海外で異なるフォーマットに整形
def get_formatted_location(latitude, longitude):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={API_KEY}&language=ja"
    response = requests.get(url)
    data = response.json()
    
    if "results" in data and len(data["results"]) > 0:
        address_components = data["results"][0]["address_components"]
        
        country = ""
        prefecture = ""
        city = ""

        for component in address_components:
            if "country" in component["types"]:
                country = component["long_name"]  # 日本 or France
            elif "administrative_area_level_1" in component["types"]:
                prefecture = component["long_name"]  # 東京都, 大阪府
            elif "locality" in component["types"] or "administrative_area_level_2" in component["types"]:
                city = component["long_name"]  # 新宿区, 横浜市

        # 国内なら「東京都 新宿区」、海外なら「フランス Paris」に変換
        if country == "日本":
            return f"{prefecture} {city}".replace("都", "").replace("府", "").replace("県", "")  # 「東京都 新宿区」→「東京 新宿」
        else:
            return f"{country} {city}"

    return "住所情報が取得できませんでした"

# Excelファイルの読み込み
df = pd.read_excel("100データ5.xlsx", engine="openpyxl")

# location_name列を追加（緯度・経度を使って変換）
df["location"] = df.apply(lambda row: get_formatted_location(row["latitude"], row["longitude"]), axis=1)

In [7]:
documents = []
for idx, row in df.iterrows():
    text_content = f"""日記ID: {row['id']}
日時: {row['timestamp']}
場所: {row['location']}
本文: {row['diary_text']}
画像: {row.get('image_path', '画像パスなし')}"""

    documents.append(Document(
        page_content=text_content,
        metadata={"id": row["id"]}
    ))

In [8]:
documents

[Document(metadata={'id': 1}, page_content='日記ID: 1\n日時: 1950-01-15T10:30:00Z\n場所: 住所情報が取得できませんでした\n本文: 父の数学の参考書を読みながら、将来の夢について考えた。野球への情熱が日に日に強くなっていく。レフトの守備練習に励み、甲子園を目指す決意を新たにした。\n画像: 画像パスなし'),
 Document(metadata={'id': 2}, page_content='日記ID: 2\n日時: 1951-03-22T15:45:00Z\n場所: 住所情報が取得できませんでした\n本文: 今日の野球の練習は厳しかったが、チームメイトと共に汗を流すことで絆が深まった。夕方は家族と一緒に夕食を楽しんだ。将来への希望に胸を膨らませている。\n画像: 画像パスなし'),
 Document(metadata={'id': 3}, page_content='日記ID: 3\n日時: 1952-05-10T09:15:00Z\n場所: 住所情報が取得できませんでした\n本文: 今朝、父と一緒に野球の戦術について語り合った。数学的思考が野球にも活かせると感じた。夕方は近所の友人とキャッチボールを楽しんだ。\n画像: 画像パスなし'),
 Document(metadata={'id': 4}, page_content='日記ID: 4\n日時: 1953-07-25T16:20:00Z\n場所: 住所情報が取得できませんでした\n本文: 夏の甲子園予選に向けて、猛暑の中で練習に励んでいる。汗と熱意で、チームの勝利を信じている。夜は冷たい麦茶で喉を潤した。\n画像: 画像パスなし'),
 Document(metadata={'id': 5}, page_content='日記ID: 5\n日時: 1954-09-05T14:40:00Z\n場所: 住所情報が取得できませんでした\n本文: 高校野球の最後の大会。悔いなく全力で戦った。負けは悔しいが、チームメイトとの絆は一生の宝物となった。夕方、父から励ましの言葉をもらった。\n画像: 画像パスなし'),
 Document(metadata={'id': 6}, page_content='日記ID: 6\n日時: 1955-

In [17]:
# 2. 埋め込みモデルを設定
# --- (b) Azure OpenAI Embeddingsを使う場合 ---
embedding_model = AzureOpenAIEmbeddings(
    deployment="text-embedding-3-large",  # ✅ ← 正しくは deployment=
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2023-05-15",
    chunk_size=1
)

In [18]:
# --- (b) Azure OpenAI Embeddingsを使う場合 ---

# FAISSベクトルストアを構築
#  - from_documents() で一度にDocumentリストをEmbeddingし、インデックスを構築
faiss_db = FAISS.from_documents(documents, embedding_model)

# =====================================
# 3) RetrievalQA チェーン構築
# =====================================
# Azure OpenAI を LangChain の llm として定義
chat_llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"), 
    openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    deployment_name="gpt-4o-mini",  # Azure でデプロイしたモデル名
    temperature=0.7
)

# RetrievalQAを作成
retrieval_qa = RetrievalQA.from_chain_type(
    llm=chat_llm,
    chain_type="stuff",  # 単純に検索結果ドキュメントを結合する方式
    retriever=faiss_db.as_retriever(search_kwargs={"k": 3})
)

C:\Users\堀江雅彦\AppData\Local\Temp\ipykernel_16496\4052380232.py:11: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  chat_llm = AzureChatOpenAI(


In [10]:
# 家族構成の定義
calling_name1="僕"
calling_name2="タラちゃん"
calling_name3="ママ"
relation_name1="本人"
relation_name2="息子"
relation_name3="妻"
birthday_name1="1940-07-21"
birthday_name2="1970-05-24"
birthday_name3="1970-10-18"

In [17]:
# --- (b) Azure OpenAI Embeddingsを使う場合 ---

# =====================================
# 4) デモ用の質問 + 類似スコア付き検索
# =====================================
question = "誕生日の思い出は？"

# スコア付きで検索（類似度が高いほどスコアは小さい）
retriever = faiss_db.as_retriever(search_kwargs={"k": 3})
docs_and_scores = retriever.vectorstore.similarity_search_with_score(question, k=3)

In [18]:
# 検索結果の表示
print("▼ 検索でヒットした日記とスコア（スコアが小さいほど類似）:")
for i, (doc, score) in enumerate(docs_and_scores):
    print(f"\n--- Top {i+1} ---")
    print(f"スコア: {score:.4f}")
    print(doc.page_content)  # 長いので最初だけ表示

# ヒットしたドキュメントを連結して、プロンプトとして LLM に渡す
context = "\n---\n".join([doc.page_content for doc, _ in docs_and_scores])

# 親として振る舞うよう明示したプロンプト
final_prompt = PromptTemplate.from_template(
    """
    あなたは天国にいて子供を見守っています。
    あなたに語りかけるあなたの子供のコメント{question}に対して、
    家族の素晴らしい思い出を一緒に回想してコメントを作成してください。
    全てのコメントは親子間のフランクな口調で、優しく語りかけるような表現にしてください。
    あなたが残した日記のコメントは下記3件あります。{context}
    日記の内容を含めて回想してください。
    ただし、子供のコメントに対して、ロケーション・場所・年月日が大きく異なる日記の内容は回想に含める必要はありません。
    なお、あなたの家族構成と呼称を伝えます。{relation_name1}のことを{calling_name1}、
    {relation_name2}のことを{calling_name2}、{relation_name3}のことを{calling_name3}と呼んでください。
    あなたの家族の誕生日は、{relation_name1}は{birthday_name1}、{relation_name2}は{birthday_name2}、
    {relation_name3}は{birthday_name3}となります。こちらも文脈に合えば使ってください。
    200文字以内にしてください。
    """
).partial(
    calling_name1=calling_name1,
    calling_name2=calling_name2,
    calling_name3=calling_name3,
    relation_name1=relation_name1,
    relation_name2=relation_name2,
    relation_name3=relation_name3,
    birthday_name1=birthday_name1,
    birthday_name2=birthday_name2,
    birthday_name3=birthday_name3
)

final_prompt_text = final_prompt.format(
    question=question,
    context=context
)

# LLM に直接問い合わせ
response = chat_llm.invoke(final_prompt_text)

print("\n▼User:")
print(question)
print("▼Assistant (Parent):")
print(response.content)

▼ 検索でヒットした日記とスコア（スコアが小さいほど類似）:

--- Top 1 ---
スコア: 0.8810
日記ID: 7
日時: 1991-03-20T14:11:33Z
場所: 東京 墨田区
本文: 娘の誕生日。手作りのケーキと手紙。妻と二人で、娘の成長と愛情に感動。家族の温かさを感じる特別な日。
画像: 画像パスなし

--- Top 2 ---
スコア: 1.1120
日記ID: 22
日時: 2000-02-14T11:33:44Z
場所: 東京 墨田区
本文: 娘が初めて手作りのバレンタインチョコレート。少し歪な形だけど、愛情たっぷりの一品。妻と目を合わせ、娘の成長に静かに感動する。
画像: 画像パスなし

--- Top 3 ---
スコア: 1.1245
日記ID: 38
日時: 2007-02-14T11:33:22Z
場所: 東京 新宿区
本文: バレンタインデー。娘が手作りのチョコレートと手紙。妻と二人で、娘の成長と愛情に感動。家族の温かさを感じる特別な日。
画像: 画像パスなし

▼User:
誕生日の思い出は？
▼Assistant (Parent):
ああ、タラちゃん、君が小さかった頃の誕生日を思い出すよ。1991年の娘の誕生日、僕とママが手作りのケーキを用意して、君の成長を見守りながら感動したね。あの温かい家族の雰囲気、今でも忘れられないよ。君の笑顔が、僕たちの宝物なんだ。これからも、その笑顔を大切にしてね。


以下はGraphRAGの作成

In [ ]:
class GraphRAG:
    def __init__(self, rag_model_name='facebook/rag-sequence-nq', graph_data=None):
        # RAGモデル用のトークナイザーをロード
        self.tokenizer = RagTokenizer.from_pretrained(rag_model_name)
        # RAGのリトリーバー(ドキュメントを検索)をロード
        self.retriever = RagRetriever.from_pretrained(rag_model_name, index_name="custom", passages_path=None)
        # テキストを生成するRAGモデルをロード
        self.model = RagSequenceForGeneration.from_pretrained(rag_model_name)
        
        # グラフデータを用意（例：ノード情報とエッジを定義）graph_dataが渡されればそれを使用し、なければ空のグラフを作成
        self.graph = graph_data or nx.Graph()

    def add_node(self, node_id, node_data):
        self.graph.add_node(node_id, data=node_data)
    
    def add_edge(self, node_id1, node_id2, edge_data):
        self.graph.add_edge(node_id1, node_id2, data=edge_data)
    
    def search_graph(self, query):
        # グラフ内で情報を検索するロジック（例：ノードの関連情報を抽出）
        nodes = list(self.graph.nodes(data=True))
        return [node for node in nodes if query.lower() in node[1]['data'].lower()]
    
    def generate_with_graph(self, input_text):
        # グラフ検索を行い、関連する情報を取得
        relevant_nodes = self.search_graph(input_text)
        context = ' '.join([node[1]['data'] for node in relevant_nodes])
        
        # RAGで情報生成
        inputs = self.tokenizer(input_text, return_tensors="pt")
        retrieved_docs = self.retriever(input_ids=inputs["input_ids"], context=context, n_docs=5)
        generated_ids = self.model.generate(input_ids=inputs["input_ids"], retrieved_doc_ids=retrieved_docs['doc_ids'])
        
        return self.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

# GraphRAGの使用例
graph_data = nx.Graph()
graph_data.add_node(1, data="AI technology in healthcare")
graph_data.add_node(2, data="Recent advancements in NLP")
graph_data.add_edge(1, 2, data="AI and NLP are closely related in healthcare.")

graph_rag = GraphRAG(graph_data=graph_data)
input_query = "Tell me about AI in healthcare"
output = graph_rag.generate_with_graph(input_query)
print(output)